In [12]:
import torch
device = "cuda:2"
model_path = '../Path/To/results/models/Food101-best-20240505-013323.pth'
model = torch.load(model_path).to(device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 47.53 GiB of which 12.44 MiB is free. Process 1955584 has 10.39 GiB memory in use. Process 2319473 has 10.39 GiB memory in use. Process 2951439 has 4.47 GiB memory in use. Process 2956205 has 4.47 GiB memory in use. Process 2959648 has 10.36 GiB memory in use. Process 2962038 has 3.96 GiB memory in use. Including non-PyTorch memory, this process has 1.11 GiB memory in use. Process 3060104 has 840.00 MiB memory in use. Process 3060974 has 840.00 MiB memory in use. Process 3061392 has 698.00 MiB memory in use. Of the allocated memory 765.83 MiB is allocated by PyTorch, and 70.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

AttributeError: 'collections.OrderedDict' object has no attribute 'named_parameters'

AttributeError: 'collections.OrderedDict' object has no attribute 'text_encoder'

odict_keys(['image_encoder.model.embeddings.cls_token', 'image_encoder.model.embeddings.position_embeddings', 'image_encoder.model.embeddings.patch_embeddings.projection.weight', 'image_encoder.model.embeddings.patch_embeddings.projection.bias', 'image_encoder.model.encoder.layer.0.attention.attention.query.weight', 'image_encoder.model.encoder.layer.0.attention.attention.query.bias', 'image_encoder.model.encoder.layer.0.attention.attention.key.weight', 'image_encoder.model.encoder.layer.0.attention.attention.key.bias', 'image_encoder.model.encoder.layer.0.attention.attention.value.weight', 'image_encoder.model.encoder.layer.0.attention.attention.value.bias', 'image_encoder.model.encoder.layer.0.attention.output.dense.weight', 'image_encoder.model.encoder.layer.0.attention.output.dense.bias', 'image_encoder.model.encoder.layer.0.intermediate.dense.weight', 'image_encoder.model.encoder.layer.0.intermediate.dense.bias', 'image_encoder.model.encoder.layer.0.output.dense.weight', 'image_en